# 行李配送问题

## 目标和前提条件

在这个例子中，您将学习如何使用数学优化来解决带时间窗口的车辆路径问题，该问题涉及帮助一家公司确定运送丢失或延误行李到其合法所有者所需的最少货车数量，并确定货车到客户的最优分配方案。

这个模型是 H. Paul Williams 所著《数学规划中的模型构建》第五版第287-289页和第343-344页的第27个例子。

这是一个高级建模示例，我们假设您了解Python和Gurobi Python API，并且具有构建数学优化模型的高级知识。通常，这些示例的目标函数和/或约束条件比较复杂，或需要使用Gurobi Python API的高级功能。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 问题描述

一家拥有6辆货车的小公司与多家航空公司签订了合同，每天晚上6点从希思罗机场接收伦敦地区客户的丢失或延误行李。合同规定每位客户必须在晚上8点前收到他们的行李。公司需要一个模型来建议他们需要使用的最少货车数量，以及每辆货车应该按什么顺序为哪些客户送货。每辆货车在实际使用中没有容量限制。每辆货车可以装载两小时内需要配送的所有行李。为了解决这个问题，我们可以制定一个优化模型，以最小化需要使用的货车数量。

## 模型构建

### 集合和索引

$i,j \in \text{位置集合} \equiv L=\{0,1..(n-1)\}$：位置集合，其中 $0$ 是单一仓库（希思罗机场）的索引，$n$ 是位置数量。

$k \in \text{货车集合} \equiv V=\{0..K-1\}$：货车的索引和集合，其中 $K$ 是货车数量。

$S_k \in S$：货车 $k$ 的路线，即该货车访问的位置子集。

### 参数

$t_{i,j} \in \mathbb{R}^+$：从位置 $i$ 到位置 $j$ 的行驶时间。

### 决策变量

$x_{i,j,k} \in \{0,1\}$：二元变量，如果货车 $k$ 直接从位置 $i$ 访问并前往位置 $j$ 则为1，否则为0。

$y_{i,k} \in \{0,1\}$：二元变量，如果货车 $k$ 访问位置 $i$ 则为1，否则为0。

$z_{k} \in \{0,1\}$：二元变量，如果使用货车 $k \in \{1,2..K\}$ 则为1，否则为0。

### 目标函数

**货车数量**：最小化使用的货车数量。

\begin{equation}
\text{最小化} \quad \sum_{k = 1}^{K} z_k
\end{equation}

### 约束条件

**货车利用率**：对于所有非仓库位置（即 $i > 0$），如果该位置被货车 $k$ 访问，则该货车被使用。

\begin{equation}
y_{i,k} \leq z_{k} \quad \forall i \in L \setminus \{0\}, \; k \in V
\end{equation}

**行驶时间**：任何货车行驶时间不超过120分钟。注意我们不考虑返回仓库的行驶时间。

\begin{equation}
\sum_{i \in L} \sum_{j \in L \setminus \{0\}} t_{i,j} \cdot x_{i,j,k} \leq 120 \quad \forall k \in V
\end{equation}

**访问所有客户**：每个客户位置恰好被一辆货车访问。

\begin{equation}
\sum_{k \in V} y_{i,k} = 1 \quad \forall i \in L \setminus \{0\}
\end{equation}

**仓库**：希思罗机场被每辆使用的货车访问。（注意：为了提高性能，我们通过分解这个约束条件与书中的版本有所不同）。

\begin{equation}
y_{0,k} = z_k \quad \forall k \in V
\end{equation}

**到达位置**：如果位置 $j$ 被货车 $k$ 访问，则该货车必须从另一个位置 $i$ 到达。

\begin{equation}
\sum_{i \in L} x_{i,j,k} = y_{j,k} \quad \forall j \in L, \; k \in V
\end{equation}

**离开位置**：如果货车 $k$ 离开位置 $j$，则该货车必须前往另一个位置 $i$。

\begin{equation}
\sum_{i \in L} x_{j,i,k} = y_{j,k} \quad \forall j \in L, \; k \in V
\end{equation}

**打破对称性**：

\begin{equation}
\sum_{i \in L} y_{i,k} \geq \sum_{i \in L} y_{i,k+1} \quad \forall k \in \{0..K-1\}
\end{equation}

**消除子回路**：这些约束确保每条货车路线中不存在环路。

\begin{equation}
\sum_{(i,j) \in S_k}x_{i,j,k} \leq |S_k|-1 \quad \forall k \in K, \; S_k \subseteq L
\end{equation}

## Python实现

我们导入Gurobi Python模块和其他Python库。

In [ ]:
%pip install gurobipy

In [1]:
import sys
import math
import random
from itertools import permutations
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0

## 输入数据
我们定义模型的所有输入数据。用户定义包括仓库在内的位置数量和货车数量。我们随机确定每个位置的坐标，然后计算每对位置之间的欧几里得距离。我们假设速度为60公里/小时，即1公里/分钟。因此行驶时间等于距离。

In [2]:
# 包括仓库在内的位置数量。仓库的索引是0
n = 17
locations = [*range(n)]

# 货车数量
K = 6
vans = [*range(K)]

# 创建n个随机点
# 仓库位于(0,0)坐标
random.seed(1)
points = [(0, 0)]
points += [(random.randint(0, 50), random.randint(0, 50)) for i in range(n-1)]

# 每对点之间的欧几里得距离字典
# 假设速度为60公里/小时，即1公里/分钟。因此行驶时间等于距离
time = {(i, j):
        math.sqrt(sum((points[i][k]-points[j][k])**2 for k in range(2)))
        for i in locations for j in locations if i != j}

## 模型部署

我们创建一个模型和变量。决策变量确定每辆货车访问客户子集的顺序、每辆货车访问哪些客户，以及是否使用某辆货车。

In [3]:
m = gp.Model('lost_luggage_distribution')

# 创建变量：

# x=1，如果货车k直接从位置i访问并前往位置j
x = m.addVars(time.keys(), vans, vtype=GRB.BINARY, name='FromToBy')

# y=1，如果客户i被货车k访问
y = m.addVars(locations, vans, vtype=GRB.BINARY, name='visitBy')

# 使用的货车数量是决策变量
z = m.addVars(vans, vtype=GRB.BINARY, name='used')

# 每辆货车的行驶时间
t = m.addVars(vans, ub=120, name='travelTime')

# 最大行驶时间
s = m.addVar(name='maxTravelTime')

Set parameter LicenseID to value 2601452


## 约束条件

对于所有非仓库位置（即 $i > 0$），如果该位置被货车 $k$ 访问，则该货车被使用。

In [4]:
# 货车使用约束
visitCustomer = m.addConstrs((y[i,k] <= z[k]  for k in vans for i in locations if i > 0), name='visitCustomer' )

任何货车行驶时间不超过120分钟。我们对原始H.P. Williams版本做了小改动，为每辆货车引入了一个行驶时间的松弛变量t[k]。

In [5]:
# 行驶时间约束
# 不包括返回仓库的时间
travelTime = m.addConstrs((gp.quicksum(time[i,j]*x[i,j,k] for i,j in time.keys() if j > 0) == t[k] for k in vans), 
                          name='travelTimeConstr' )

每个客户位置恰好被一辆货车访问

In [6]:
# 访问所有客户
visitAll = m.addConstrs((y.sum(i,'*') == 1 for i in locations if i > 0), name='visitAll' )

希思罗机场（仓库）被每辆使用的货车访问。

In [7]:
# 仓库约束
depotConstr = m.addConstrs((y[0, k] == z[k] for k in vans), name='depotConstr' )

如果位置 j 被货车 k 访问，则该货车必须从另一个位置 i 到达。

In [8]:
# 到达客户位置约束
ArriveConstr = m.addConstrs((x.sum('*',j,k) == y[j,k] for j,k in y.keys()), name='ArriveConstr' )

如果货车 k 离开位置 j，则该货车必须前往另一个位置 i。

In [9]:
# 离开客户位置约束
LeaveConstr = m.addConstrs((x.sum(j,'*',k) == y[j,k] for j,k in y.keys()), name='LeaveConstr' )

打破对称性约束。

In [10]:
breakSymm = m.addConstrs((y.sum('*',k-1) >= y.sum('*',k) for k in vans if k>0), name='breakSymm' )

将最大行驶时间与每辆货车的行驶时间关联起来

In [11]:
maxTravelTime = m.addConstrs((t[k] <= s for k in vans), name='maxTravelTimeConstr')

# Alternately, as a general constraint:
# maxTravelTime = m.addConstr(s == gp.max_(t), name='maxTravelTimeConstr')

### 目标函数
我们使用两个层次的目标：
- 首先，最小化使用的货车数量
- 然后，最小化时间限制约束的最大值

In [12]:
m.ModelSense = GRB.MINIMIZE
m.setObjectiveN(z.sum(), 0, priority=1, name="Number of vans")
m.setObjectiveN(s, 1, priority=0, name="Travel time")

### 回调函数定义
子回路约束防止货车在不经过希思罗机场（起点或终点）的情况下访问一组目的地。由于这些约束的数量呈指数级增长，我们不想将它们全部添加到模型中。相反，我们使用回调函数来找到违反的子回路约束，并将它们作为延迟约束添加到模型中。

In [13]:
# 回调函数 - 使用延迟约束消除子回路
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # 创建解中选择的边的列表
        vals = model.cbGetSolution(model._x)
        selected = gp.tuplelist((i,j) for i, j, k in model._x.keys()
                                if vals[i, j, k] > 0.5)
        # 在选定的边列表中找到最短环路
        tour = subtour(selected)
        if len(tour) < n: 
            for k in vans:
                model.cbLazy(gp.quicksum(model._x[i, j, k]
                                         for i, j in permutations(tour, 2))
                             <= len(tour)-1)


# 给定边的元组列表，找出不包含仓库(0)的最短子回路
def subtour(edges):
    unvisited = list(range(1, n))
    cycle = range(n+1)  # 初始长度多一个城市
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            if current != 0:
                unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j == 0 or j in unvisited]
        if 0 not in thiscycle and len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

## 求解模型

In [14]:
# 验证模型公式
m.write('lost_luggage_distribution.lp')

# 运行优化引擎
m._x = x
m.Params.LazyConstraints = 1
m.optimize(subtourelim)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Non-default parameters:
LazyConstraints  1

Optimize a model with 339 rows, 1747 columns and 5492 nonzeros
Model fingerprint: 0x06e2dd4c
Variable types: 7 continuous, 1740 integer (1740 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve...
---------------------------------------------------------------------------

Presolve removed 22 rows an

## 分析

下面是每辆使用的货车的最优路线和总行李配送时间报告。

In [15]:
# 打印最优路线
for k in vans:
    route = gp.tuplelist((i,j) for i,j in time.keys() if x[i,j,k].X > 0.5)
    if route:
        i = 0
        print(f"货车 {k} 的路线: {i}", end='')
        while True:
            i = route.select(i, '*')[0][1]
            print(f" -> {i}", end='')
            if i == 0:
                break
        print(f". 行驶时间: {round(t[k].X,2)} 分钟")

print(f"最大行驶时间: {round(s.X,2)}")

货车 0 的路线: 0 -> 3 -> 16 -> 15 -> 2 -> 7 -> 14 -> 6 -> 5 -> 13 -> 0. 行驶时间: 105.42 分钟
货车 1 的路线: 0 -> 9 -> 8 -> 1 -> 12 -> 10 -> 4 -> 11 -> 0. 行驶时间: 104.78 分钟
最大行驶时间: 105.42


## 参考文献

H. Paul Williams，《数学规划中的模型构建》，第五版。

Copyright © 2020 Gurobi Optimization, LLC